In [1]:
#===============================================================================
# PAKETE LADEN
#===============================================================================

#!pip uninstall -y scipy
#!pip uninstall -y gensim
#!pip install scipy == 1.10.1
#!pip install gensim

#Schneidjerberg
import re
import os
import spacy
import shutil
import pandas as pd
import numpy as np
import gensim

#NLP Pakekte importieren (Linguistische Sachen)
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
#Importiert Befehle, die für die Arbeit mit Word wichtig sind
!pip install python-docx
from docx import Document
import shutil

#Lemmatisierung
!pip install spacy
!python -m spacy download de_core_news_sm

#Tokenisierung
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt', download_dir='/usr/local/share/nltk_data')
nltk.download('punkt_tab')

#Importiert Befehle, welche mit dem Drive zu tun haben
from google.colab import drive

#Topic Model
!pip install pyLDAvis
from gensim import corpora
import pyLDAvis.gensim_models
from gensim.models import TfidfModel, LdaMulticore
from gensim.similarities import MatrixSimilarity
from gensim.models.coherencemodel import CoherenceModel

#===============================================================================
# VERBINDUNG ZUM GOOGLE DRIVE & ORDNERVERWALTUNG
#===============================================================================

#Stellt eine Verbindung zum Drive her
drive.mount('/content/drive')

#Definiert die Variable "Word-Dateien", welche auf den Ordner "Zeitungsartikel"
#  Pfad zu deinem Ordner auf Google Drive
folder_path = '/content/drive/MyDrive/Zeitungsartikel_final'

#===============================================================================
# unnötige Txt-Dokumente löschen
#===============================================================================

# Funktion zum Löschen der .txt-Dateien im Ordner und seinen Unterordnern
def delete_txt_files_in_directory(folder_path):
    # Durchlaufe alle Dateien und Unterordner im angegebenen Ordner
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            if filename.endswith('.txt'):  # Nur .txt-Dateien löschen
                file_path = os.path.join(dirpath, filename)
                try:
                    os.remove(file_path)  # Lösche die Datei
                    #print(f"Datei gelöscht: {file_path}")
                except Exception as e:
                    print(f"Fehler beim Löschen der Datei {file_path}: {e}")

delete_txt_files_in_directory(folder_path)

#===============================================================================
# DATEIEN UMBENENNEN (Leerzeichen entfernen)
#===============================================================================
# Funktion zur Bereinigung der Dateinamen
def bereinige_dateinamen(folder_path):
    if not os.path.exists(folder_path):
        raise FileNotFoundError("Der Ordner wurde nicht gefunden.")

    for root, _, files in os.walk(folder_path):
        for datei in files:
            if datei.lower().endswith(".docx"):
                alter_name = os.path.join(root, datei)
                neuer_name = os.path.join(root, datei.replace(" ", "_").lower())

                try:
                    os.rename(alter_name, neuer_name)
                    #print(f"Datei umbenannt: {alter_name} -> {neuer_name}")
                except Exception as e:
                    print(f"Fehler beim Umbenennen der Datei {datei}: {e}")

bereinige_dateinamen(folder_path)

#===============================================================================
#Umwandlung von Word in txt
#===============================================================================
# Funktion, um .docx-Dateien in .txt-Dateien umzuwandeln
def docx_to_txt(docx_path, txt_path):
    doc = Document(docx_path)
    with open(txt_path, 'w', encoding='utf-8') as txt_file:
        for para in doc.paragraphs:
            txt_file.write(para.text + '\n')

# Alle .docx-Dateien im angegebenen Ordner und allen Unterordnern durchsuchen und umwandeln
for dirpath, dirnames, filenames in os.walk(folder_path):
    for filename in filenames:
        if filename.endswith('.docx'):  # Jetzt nur auf 'filename' anwenden
            docx_path = os.path.join(dirpath, filename)  # Voller Pfad zur .docx-Datei
            txt_filename = filename.replace('.docx', '.txt')
            txt_path = os.path.join(dirpath, txt_filename)  # Voller Pfad zur .txt-Datei

            docx_to_txt(docx_path, txt_path)

#===============================================================================
#Dokumente systematisch einlesen + Reduzierung auf Textbody
#===============================================================================
# Funktion, die den Text zwischen 'Body' und 'Classification' extrahiert
def extract_body_classification(text):
    start_word = "Body"
    end_word = "Classification"

    # Suche nach dem Start- und Endpunkt des Textes
    start_index = text.find(start_word)
    end_index = text.find(end_word)

    if start_index != -1 and end_index != -1 and start_index < end_index:
        # Extrahiere den Text zwischen den beiden Wörtern
        extracted_text = text[start_index + len(start_word):end_index].strip()
        return extracted_text
    else:
        return None

# Hauptfunktion, die durch alle Unterordner und Dateien geht
def process_txt_files(folder_path):
    # Durchlaufe alle Dateien und Unterordner im angegebenen Ordner
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            if filename.endswith('.txt'):  # Nur .txt-Dateien bearbeiten
                file_path = os.path.join(dirpath, filename)

                # Öffne die Datei zum Lesen
                with open(file_path, 'r', encoding='utf-8') as file:
                    text = file.read()

                # Extrahiere den Text zwischen 'Body' und 'Classification'
                extracted_text = extract_body_classification(text)

                if extracted_text:
                    # Speichere den extrahierten Text zurück in die Datei
                    with open(file_path, 'w', encoding='utf-8') as file:
                        file.write(extracted_text)
                    #print(f"Datei {filename} wurde bearbeitet und bereinigt.")
                else:
                    print(f"Kein Text zwischen 'Body' und 'Classification' in {filename} gefunden.")

# Starte die Verarbeitung
process_txt_files(folder_path)

#===============================================================================
# DATEN BEREINIGEN: Tokenisierung
#===============================================================================

# Funktion zur Tokenisierung einer Datei
def tokenize_text(text):
    # Umwandlung in Kleinbuchstaben
    text = text.lower()
    # Entfernen von Sonderzeichen und Zahlen mit regulären Ausdrücken
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenisierung des Textes
    tokens = word_tokenize(text)
    return tokens

# Funktion zur Verarbeitung aller .txt-Dateien in einem Ordner und dessen Unterordnern
def process_files_in_directory(folder_path):
    all_tokens = []

    # Durchlaufe alle Dateien und Unterordner
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                # Lese die Datei
                with open(file_path, 'r', encoding='utf-8') as f:
                    text = f.read()
                    # Tokenisiere den Text und füge die Tokens der Liste hinzu
                    tokens = tokenize_text(text)
                    all_tokens.extend(tokens)

    return all_tokens

# Beispiel: Verzeichnis, das verarbeitet werden soll
tokens = process_files_in_directory(folder_path)

#===============================================================================
# DATEN BEREINIGEN: Lemmatisierung
#===============================================================================
# Lade das deutsche Sprachmodell von spaCy
nlp = spacy.load('de_core_news_sm')

# Funktion zur Lemmatisierung eines Textes
def lemmatize_text(text):
    # Verarbeite den Text mit spaCy
    doc = nlp(text)
    # Extrahiere die Lemmata der Tokens
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    return lemmatized_text

# Funktion, die rekursiv alle .txt-Dateien im Ordner und seinen Unterordnern durchsucht
def lemmatize_files_in_directory(directory):
    # Durchlaufe alle Dateien und Unterordner im angegebenen Ordner
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.txt'):  # Wenn es sich um eine .txt-Datei handelt
                file_path = os.path.join(root, file)
                try:
                    # Öffne die Datei und lese den Inhalt
                    with open(file_path, 'r', encoding='utf-8') as f:
                        text = f.read()

                    # Lemmatisiere den Text
                    lemmatized_text = lemmatize_text(text)

                    # Überschreibe die Originaldatei mit dem lemmatisierten Text
                    with open(file_path, 'w', encoding='utf-8') as f:
                        f.write(lemmatized_text)

                except Exception as e:
                    print(f"Fehler beim Verarbeiten von {file_path}: {e}")

lemmatize_files_in_directory(folder_path)

#===============================================================================
# DATEN BEREINIGEN: Stopwörter
#===============================================================================

# Liste der Stopwörter (kann beliebig erweitert werden)
stopwoerter = ['aber', 'alle', 'allem', 'allen', 'aller', 'alles', 'als', 'also', 'am', 'an', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'anderes', 'anderm',
               'andern', 'anderr', 'anders', 'auch', 'auf', 'aus', 'bei', 'bin', 'bis', 'bist', 'da', 'damit', 'dann', 'der', 'den', 'des', 'dem', 'die', 'das', 'dass',
               'daß', 'derselbe', 'derselben', 'denselben', 'desselben', 'demselben', 'dieselbe', 'dieselben', 'dasselbe', 'dazu', 'dein', 'deine', 'deinem', 'deinen',
               'deiner', 'deines', 'denn', 'derer', 'dessen', 'dich', 'dir', 'du', 'dies', 'diese', 'diesem', 'diesen', 'dieser', 'dieses', 'doch', 'dort', 'durch', 'ein',
               'eine', 'einem', 'einen', 'einer', 'eines', 'einig', 'einige', 'einigem', 'einigen', 'einiger', 'einiges', 'einmal', 'er', 'ihn', 'ihm', 'es',
               'etwas', 'euer', 'eure', 'eurem', 'euren', 'eurer', 'eures', 'für', 'gegen', 'gewesen', 'hab', 'habe', 'haben', 'hat', 'hatte', 'hatten', 'hier',
               'hin', 'hinter', 'ich', 'mich', 'mir', 'ihr', 'ihre', 'ihrem', 'ihren', 'ihrer', 'ihres', 'euch', 'im', 'in', 'indem', 'ins', 'ist', 'jede', 'jedem',
               'jeden', 'jeder', 'jedes', 'jene', 'jenem', 'jenen', 'jener', 'jenes', 'jetzt', 'kann', 'kein', 'keine', 'keinem', 'keinen', 'keiner', 'keines',
               'können', 'könnte', 'machen', 'man', 'manche', 'manchem', 'manchen', 'mancher', 'manches', 'mein', 'meine', 'meinem', 'meinen', 'meiner',
               'meines', 'mit', 'muss', 'musste', 'nach', 'nicht', 'nichts', 'noch', 'nun', 'nur', 'ob', 'oder', 'ohne', 'sehr', 'sein', 'seine', 'seinem',
               'seinen', 'seiner', 'seines', 'selbst', 'sich', 'sie', 'ihnen', 'sind', 'so', 'solche', 'solchem', 'solchen', 'solcher', 'solches', 'soll',
               'sollte', 'sondern', 'sonst', 'über', 'um', 'und', 'uns', 'unsere', 'unserem', 'unseren', 'unser', 'unseres', 'unter', 'viel', 'vom', 'von',
               'vor', 'während', 'war', 'waren', 'warst', 'was', 'weg', 'weil', 'weiter', 'welche', 'welchem', 'welchen', 'welcher', 'welches', 'wenn', 'werde',
               'werden', 'wie', 'wieder', 'will', 'wir', 'wird', 'wirst', 'wo', 'wollen', 'wollte', 'würde', 'würden', 'zu', 'zum', 'zur', 'zwar', 'zwischen',
               'werden', 'sein', 'haben', 'sagen', 'geben', 'kommen', 'können', 'stehen', 'müssen', 'gehen', 'sagen', 'vieler', 'schon', 'gut', 'der', 'ganz',
               'seit', 'deshalb', 'besonders', ':', '--', 'davon', 'sehen', 'etwa', 'ja', 'dafür', 'erst', 'vielleicht', 'bringen', 'wer', 'hören', 'holen',
               'fahren', 'liegen', 'mal', 'ab', 'allerdings','GesamtseitenPDF', '%','*','Gesamtseiten','PDF', 'Gesamtseiten-PDF', 'Original','sollen', 'mehr', 'Gesamtseiten--PDF', 'taz',
               'welt', 'WELT', 'Zeit', 'Bild', 'zwei','drei', 'vier', 'erster', 'zweiter', 'dritter', 'ZEIT','gesamtseiten-pdf', '%', '*', 'Gesamtseiten', 'PDF', 'Gesamtseiten-PDF', 'Original', 'original', 'sollen', 'mehr', 'Gesamtseiten--PDF', 'taz',
               'Köhler', 'köhler', 'angelina', 'Angelina', 'Fox', 'fox' 'Solomon', 'solomon','Jahr', 'jahr', 'groß', 'neu','immer','hoch','heute','weniger','zeigen','natürlich','dürfen','Teil','stellen','vergangen', 'groß',
               'bleiben', 'stark','bereits','Tag','wegen','steigen','halten','letzter','weit','stellen','dürfen', 'mensch',

               'eigen', 'tun', 'führen', 'wissen', 'deutlich', 'lassen', 'klein', 'Klein', 'finden', 'rund', 'alt', 'gerade', 'nehmen', 'ziehen', 'oft', 'dabei', 'gestern',
               'mehrere', 'Teil', 'Dylan', 'Naomi', 'Schlotterbeck', 'dabei', 'Bickhardt', 'fast', 'Fuhrmann', 'Fischer', 'The', 'ZEIT', 'Zeit', 'zeit', 'zeit.', 'brauchen',
               'kühne', 'tag', 'dylan', 'frau', 'wichtig', 'erklären', 's', 'seite', 'art', 'schlotterbeck', 'fischer', 'orth', 'fallen', 'aktuell', 'heißen', 'pro', 'schnell',
               'erreichen', '1.', 'erster', 'heino', 'bild', 'aserbaidschan', 'bergkarabach', 'armeinen', 'aserbaidschanisch', 'haien', 'jersey', 'P.', 'Iran', 'Bickhardt','wasser','Wasser']

def remove_stopwords_from_txt(folder_path, stopwoerter):

    # Überprüfen, ob der Ordner existiert
    if not os.path.exists(folder_path):
        print("Der angegebene Ordner existiert nicht.")
        return

    # Alle Dateien im Ordner und Unterordnern durchlaufen
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # Überprüfen, ob die Datei eine .txt-Datei ist
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)

                # Datei öffnen und lesen
                with open(file_path, "r", encoding="utf-8") as f:
                    text = f.read()

                # Wörter entfernen, die in der stopwoerter-Liste enthalten sind
                text_words = text.split()
                cleaned_words = [word for word in text_words if word.lower() not in stopwoerter]

                # Bereinigten Text zusammenfügen
                cleaned_text = " ".join(cleaned_words)

                # Datei mit bereinigtem Inhalt neu speichern
                with open(file_path, "w", encoding="utf-8") as f:
                    f.write(cleaned_text)

    print("Die Stopwörter wurden aus allen .txt-Dokumenten entfernt.")

remove_stopwords_from_txt(folder_path, stopwoerter)

def remove_hyphens_from_txt(folder_path):

    # Überprüfen, ob der Ordner existiert
    if not os.path.exists(folder_path):
        print("Der angegebene Ordner existiert nicht.")
        return

    # Alle Dateien im Ordner und Unterordnern durchlaufen
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # Überprüfen, ob die Datei eine .txt-Datei ist
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)

                # Datei öffnen und lesen
                with open(file_path, "r", encoding="utf-8") as f:
                    text = f.read()

                # Sonderzeichen "-" entfernen
                cleaned_text = text.replace("-", "")

                # Datei mit bereinigtem Inhalt neu speichern
                with open(file_path, "w", encoding="utf-8") as f:
                    f.write(cleaned_text)

    print("Das Sonderzeichen '-' wurde aus allen .txt-Dokumenten entfernt.")

remove_hyphens_from_txt(folder_path)

#===============================================================================
# Regulärer Ausdruck
#===============================================================================

# Liste der Risikobegriffe (Teilworte)
risiko_keywords = ["risik", "wahrscheinlich", "poten", "eintritt", "eingetreten", "ausmaß", "schad", "schäd", "erwart", "verletz", "schutz", "vulnerab",
                   "katastroph", "natur", "bewältig", "anpass", "angepasst", "tot", "gestorben", "sterb", "ausgesetzt", "aussetz", "verlust", "verlier", "verlor",
                   "fehler", "extrem", "knapp", "verschmutz", "klima", "bedroh", "schutz", "sozi", "technologi", "ökonom","gesundheit", "umwelt", "maßnahme",
                   "schuld", "fehler", "zukunft", "zukünft", "vorhersag", "einschätz", "unsicher", "gefahr", "gefähr", "individ", "privat", "politi", "bund",
                   "region", "kommun", "lokal", "versicher", "prävent",]


def count_risk_tokens(text, risk_keywords):
    #Zählt, wie oft Risikobegriffe im Text vorkommen
    count = sum(len(re.findall(rf"{keyword}", text, re.IGNORECASE)) for keyword in risk_keywords)
    return count

def process_risk_documents_in_directory(folder_path, risk_keywords, min_risk_count=5):
    total_files = 0
    relevant_documents = []

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.txt'):
                total_files += 1
                file_path = os.path.join(root, file)

                with open(file_path, 'r', encoding='utf-8') as f:
                    text = f.read()

                risk_token_count = count_risk_tokens(text, risk_keywords)

                if risk_token_count >= min_risk_count:
                    relevant_documents.append((file_path, text))

    print(f"Gesamtzahl der .txt-Dateien: {total_files}")
    print(f"Anzahl der relevanten Dokumente nach Filterung: {len(relevant_documents)}")

    return relevant_documents

relevant_docs = process_risk_documents_in_directory(folder_path, risiko_keywords, min_risk_count=5)

#===============================================================================
#Bag of Words
#===============================================================================

from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd


# Funktion zur Erstellung der Bag-of-Words-Darstellung
def create_bag_of_words(documents):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(documents)
    feature_names = vectorizer.get_feature_names_out()
    df_bow = pd.DataFrame(X.toarray(), columns=feature_names)
    return df_bow

#===============================================================================
#Topic Model
#===============================================================================

# Funktion zum Laden der Textdateien
def load_documents_from_folder(folder_path):
    documents = []
    for subdir, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".txt"):  # Falls deine Dateien .txt sind
                file_path = os.path.join(subdir, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    documents.append(f.read())
    return documents

# Lade die Dokumente
documents = load_documents_from_folder(folder_path)

processed_docs = [text.split() for text in documents]

# Erstelle ein Dictionary und ein Korpus
dictionary = corpora.Dictionary(processed_docs)
corpus = [dictionary.doc2bow(text) for text in processed_docs]

# Erstelle das LDA Modell
lda_model = gensim.models.LdaMulticore(
    corpus,
    num_topics=4,
    id2word=dictionary,
    passes=50,
    workers=2,
    alpha=0.2,
    eta=0.1,
    iterations=800,
    random_state=99
)

#==============================================================================
# Topic Model Optimierung
#===============================================================================

# Kohärenz
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

#===============================================================================
#Visualisierung Topic Model
#===============================================================================

# Visualisiere das LDA Modell
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis


ModuleNotFoundError: No module named 'gensim'